# Group Members
Pierre Mercatoris -- Sergio Gámez Ruiz de Olano -- Pablo Bordons Estrada -- Mohammadmehdi Fayazbakhsh

# Exercise 1
Obtain the general formulation for the Google Adwords problem (described within slides 23-26 of the Linear Programming topic).

**Data:**

$a_i = $ maximum budget for company

$b_j = $ maximum of expected request for query

$c_{ij} = $ cost per add/click

**Variables:**

$x_{ij} = $ Number of adds assigned per query and company

$$\max_{x_{ij}} \quad \sum_{i=1}^n \sum_{j=1}^m c_{ij} x_{ij}$$

$$s.t. \quad \sum_{i=1}^n x_{ij} \leq b_j \quad \forall \, j$$
$$\sum_{j=1}^m c_{ij} x_{ij} \leq a_i \quad \forall \, i$$
$$x_{ij} \geq 0 \quad \forall \, i,j$$

# Exercise 2
Obtain its equivalent standard form.

$$ \min_x \quad z = c^{\top}x $$
$$s.t. \quad Ax = b $$
$$x \geq 0 $$

where $x \in {\rm I\!R}^n, c \in {\rm I\!R}^n, A \in {\rm I\!R}^{m\times n}$ and $ b \in {\rm I\!R}^m$

$$c=(1,0.75,5,0.5,0.5,2,0.5,3,1,0,0,0,0,0,0)^\top$$
$$x=(x_{A1}, x_{A2}, x_{A3},x_{B1}, x_{B2}, x_{B3},x_{C1}, x_{C2}, x_{C3}, z_1, z_2,z_3,z_4,z_5,z_6 )$$

$$x_{A1} + 0.75 x_{A2} + 5 x_{A3} +z_1 = 200 $$
$$0.5x_{B1} + 0.5 x_{B2} + 2 x_{B3} +z_2 = 150 $$
$$0.5x_{C1} + 3 x_{C2} + x_{C3} +z_3 = 180 $$
$$x_{A1}+x_{B1}+x_{C1}+z_4=150$$
$$x_{A2}+x_{B2}+x_{C2}+z_5=90$$
$$x_{A3}+x_{B3}+x_{C3}+z_6=80$$

$$
A =\left| \begin{array}{ccccccccccccccc}
1 & 0.75 & 5 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 &0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0.5 & 0.5 & 2 & 0 & 0 & 0 & 0 & 1 &0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0.5 & 3 & 1 & 0 & 0 & 1 & 0 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 &0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 &0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 &0 & 0 & 0 & 1 \\
\end{array} \right|
$$

# Exercise 3
Implement the model derived in 2) in Pyomo and solve it for the case of 10 companies and 10 queries (make up the data to be reasonable). Compute the sensitivities associated to each constraint. Report the results.

In [20]:
%%writefile 1st_assignement3.py

#structure of the code from https://github.com/Pyomo/PyomoGettingStarted/blob/master/01_PyomoOverview.ipynb

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

model.m = Param(within=NonNegativeIntegers) # Number of queries
model.n = Param(within=NonNegativeIntegers) # Number of companies

model.Ia = RangeSet(1, model.m+model.n)
model.Ja = RangeSet(1, (model.m*model.n) + (model.m+model.n))
model.I = RangeSet(1,model.n)
model.J = RangeSet(1,model.m)

model.a = Param(model.Ia, model.Ja)
model.b=Param(model.Ia) #Budget
model.c = Param(model.Ja)

model.x = Var(model.Ja, domain=NonNegativeReals)

def obj_expression(model):
    return summation(model.c, model.x)

#std form - minimize
model.OBJ = Objective(rule=obj_expression)


def ax_constraint_rule(model, i):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j] for j in model.Ja) == model.b[i]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.Ia, rule=ax_constraint_rule)


Overwriting 1st_assignement3.py


In [41]:
%%writefile 1st_assignement3.dat


param n :=10; #Companies
param m :=10; #Queries
    
param c := #revenue - as negative for the std form
    1 -1.4
    2 -4.2
    3 -2.1
    4 -0.5
    5 -0.5
    6 -2
    7 -0.5 
    8 -3
    9 -1
    10 0
    11 0
    12 0
    13 0
    14 0
    15 0
    16 0
    17 0
    18 0
    19 0
    20 0
    ;
       
param b:=
    1 179
    2 118
    3 126
    4 150
    5 90
    6 80
    7 151
    8 157
    9 92
    10 197
    11 168
    12 163
    13 187
    14 97
    15 192
    16 149
    17 127
    18 178
    19 112
    20 187
    ;
    

model.z = Param()

data.load(filename='matrix.csv', param=model.z)

param a: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 :=
1 1 0.75 5 0 0 0 0 0 0 1 0 0 0 0 0
2 0 0 0 0.5 0.5 2 0 0 0 0 1 0 0 0 0
3 0 0 0 0 0 0 0.5 3 1 0 0 1 0 0 0
4 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0
5 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0
6 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1
;

Overwriting 1st_assignement3.dat


In [48]:
for i in xrange(100):
    print '%04.3f' % random.uniform(0, 5)

1.672
2.817
3.062
1.228
0.805
4.382
1.481
1.680
1.379
0.201
3.124
1.072
3.278
1.278
1.313
0.255
2.821
0.406
4.544
3.140
0.011
3.792
2.764
2.600
4.748
1.294
4.219
2.029
0.560
4.665
4.001
2.972
1.092
4.330
0.535
1.260
4.740
4.798
1.039
1.984
0.912
3.695
1.429
1.916
1.467
4.265
3.953
3.274
1.789
3.576
2.953
3.628
2.563
1.490
3.701
1.189
4.781
0.954
4.891
2.904
0.552
4.387
1.367
2.762
0.452
4.860
4.344
1.300
0.599
3.919
4.368
4.735
1.581
0.923
0.613
0.551
0.611
4.957
0.086
1.857
4.871
1.819
3.495
2.296
0.716
3.341
0.276
2.009
0.974
4.284
3.518
2.441
3.741
0.305
1.233
2.242
1.978
2.830
2.436
0.598


In [22]:
!pyomo solve 1st_assignement3.py 1st_assignement3.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.02] Applying solver
[    0.03] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: -530.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=15, Index=Ja
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :     0 :  None : False : False : NonNegativeReals
          2 :     0 :     0 :  None : False : False : NonNegativeReals
          3 :     0 :    40 :  None : False : False : NonNegativeReals
          4 :     0 :   110 :  None : False : False : NonNegativeReals
          5 :     0 :    30 :  None : False : False : NonNegativeReals
          6 :     0 :    40 :  None : False : False : NonNegativeReals
          7 :     0 :     0 :  None : False : False : NonNegativeReals
          8 :     0 :    60 :  None : False

In [23]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -530.0
  Upper bound: -530.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 16
  Number of nonzeros: 25
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Error rc: 0
  Time: 0.00387406349182
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message

# Exercise 4
Given a linear programming problem in standard form:
$$\min_{x} \quad z_P = c^{\top} x $$ 
$$s.t. \quad Ax = b $$
$$x \geq 0 $$
we can define its dual problem as 
$$\max_{y} \quad z_D = b^{\top} y $$
$$s.t. \quad A^{\top}y\geq c $$
where $y$ is called teh dual variable vector.

Considering this, formulate the dual problem associated to the model derived in 2) (check slide 60 of the Linear Programming topic for an example of this transformation).

$$ \max_y \quad Z_D = [200, 150, 180, 150, 90, 80] \times y \\
= 200 y_1 + 150 y_2 + 180 y_3 + 150 y_4 + 90 y_5 + 80 y_6
$$

$$
A^{\top} =\left| \begin{array}{cccccc}
1 & 0 & 0 & 1 & 0 & 0 \\
0.75 & 0 & 0 & 0 & 1 & 0 \\
5 & 0 & 0 & 0 & 0 & 1 \\
0 & 0.5 & 0 & 1 & 0 & 0 \\
0 & 0.5 & 0 & 0 & 1 & 0 \\
0 & 2 & 0 & 0 & 0 & 1 \\
0 & 0 & 0.5 & 1 & 0 & 0 \\
0 & 0 & 3 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 \\
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
\end{array} \right|
$$

$$ A^{\top} \, y \leq -c $$

$$
y_1 + y_4 \leq -1 \\
0.75\, y_1 + y_5 \leq -0.75 \\
5\, y_1 + y_6  \leq -5 \\
0.5 \, y_2 + y_4 \leq -0.5 \\
0.5 \, y_2 + y_5 \leq -0.5 \\
2\, y_2 + y_6 \leq -2 \\
0.5\, y_3 + y_4 \leq -0.5 \\
3 \, y_3 + y_5 \leq -3 \\
y_3 + y_6 \leq -1 \\
$$

$$ y_1, y_2,y_3,y_4,y_5,y_6, \leq 0 $$

# Exercise 5
Implement the dual model derived in 4) in Pyomo and solve it for the same data in 3). Report the results.

In [34]:
%%writefile 1st_assignement3_dual.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

model.m = Param(within=NonNegativeIntegers)
model.n = Param(within=NonNegativeIntegers)

model.Ia = RangeSet(1, model.m+model.n)
model.Ja = RangeSet(1, (model.m*model.n) + (model.m+model.n))

model.a = Param(model.Ia, model.Ja)
model.b = Param(model.Ia)
model.c = Param(model.Ja)

# the next line declares a variable indexed by the set J
model.y = Var(model.Ia, domain=Reals)

#maximize
def obj_expression(model):
    return summation(model.b, model.y)

model.OBJ = Objective(rule=obj_expression, sense = maximize)

#we need to do At*y, but changing the indexes to 
#access the matrix does the trick
def ax_constraint_rule(model, j):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.y[i] for i in model.Ia) <= model.c[j]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.Ja, rule=ax_constraint_rule)

Overwriting 1st_assignement3_dual.py


In [35]:
!pyomo solve 1st_assignement3_dual.py 1st_assignement3.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.02] Applying solver
[    0.03] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: -530
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    y : Size=6, Index=Ia
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :    -1 :  None : False : False :  Reals
          2 :  None :    -1 :  None : False : False :  Reals
          3 :  None :    -1 :  None : False : False :  Reals
          4 :  None :     0 :  None : False : False :  Reals
          5 :  None :     0 :  None : False : False :  Reals
          6 :  None :     0 :  None : False : False :  Reals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  -530

  Constraints:
    AxbConstraint : Size=15
        Key : Lo

In [36]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -530.0
  Upper bound: -530.0
  Number of objectives: 1
  Number of constraints: 16
  Number of variables: 7
  Number of nonzeros: 25
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Error rc: 0
  Time: 0.00365781784058
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message

# Exercise 6
The Strong Duality Theorem states that:

If $x^*$ is the optimal solution of the primal minimization LP problem, and $y^*$ is the dual optimal solution of the corresponding dual maximization LP problem, then

$$z^*_D = b^{\top} y^* = c^{\top} x^* = z^*_P $$

Comparing the solutions in 3) and 5), check if the Strong Duality Theorem holds. What is the relationship between the sensitivities computed in 3) and the optimal value of the dual variables obtained in 5)?

The theorem holds, as both objective functions return -530.
The sensitivity in 3 equals the optimal values for y in 5 thats the relationship

# Exercise 7
Imagine now that google is able to display simultaneously n company ads for each requested query (instead of only one). Moreover, assume that the specific order in which these ads are displayed is important. Indicate how the model in 1) would need to be modified to account for these facts. Also, indicate if any additional problem data will be necessary for this new setting.

We can now show N advertisments. We go from Q*C decisions to Q*C*N decision.
we now need to know what is the revenue if a company appears in the N position queries and budget constraints stays the same.
If we set our X decisions as a vector, we get a vector with Q*C*N elements, being the first Q*C elements, the decision for N=1. (similar to points 1-6)
This is a quite dummy way to implement it, as it does not have a if company A appears 1st, 
do not show it 2nd mechanism. buuuut thats as far as i can go linearly 
(unless que build a Q*C*N*(C^N) matrix, in which we have all possible combinations of N-1 
previous advertised companies, and the expected revenues for each scenario)
I think the Q*C*N scenario would do the trick, adding some heurisits on top of it, like, do not show 
the same company twice

# IMPORTANT:

Upload the formulations 1), 2), 4) and 7) as well as the answers to the different questions in a pdf file (generated with MSWord, latex or similar) and the codes for 3) and 5) as separated .py and .dat files.

This assignment can be done individually or in groups (up to 4 components). Groups must be the same for all the assignments. Files need to be uploaded by only one member of the group. Do not forget to indicate in the pdf file all the group’s components.